In [15]:
from datetime import datetime
import json
import os

import requests
import pandas as pd

# Input basic information about the Mobius3D server
serverURL = 'http://mobius3d/'
username = 'jm84u'
password = 'Putchy123!'

# Start a session and download all the requests list in JSON format
s = requests.Session()
login = {'username': username, 'password': password}
s.post('{}auth/login'.format(serverURL), login)

<Response [200]>

In [26]:
plan_list = s.get('{}_plan/list?limit=1'.format(serverURL)).json()
with open(os.path.join('output', 'plan_list.json'), 'w') as f:
    f.write(json.dumps(plan_list, indent=4))

In [28]:
for pt in plan_list['patients']:
    for plan_chk in pt['plans']:
        request_id = plan_chk['request_cid']
        if 'linac_tps' not in request_id:
            continue
            
        plan_chk_details = s.get('{}check/details/{}?format=json'.format(serverURL, request_id)).json()
        if plan_chk_details['request']['finished'] != 'clean':
            continue
        
        plan_chk_filepath = os.path.join('output', request_id)
        try:
            os.makedirs(plan_chk_filepath)
        except FileExistsError:
            pass
        finally:
            with open(os.path.join(plan_chk_filepath, 'plan_check_details.json'), 'w') as f:
                f.write(json.dumps(plan_chk_details, indent=4))
            filename_list = s.get('{}check/details/{}/data?format=json'.format(serverURL, request_id)).json()
            for filename_info in filename_list['data']:
                file_ext = filename_info['content_type'].split('/')[1]
                filename = filename_info['filename']
                file = s.get('{}check/attachment/{}/{}'.format(serverURL, request_id, filename))
                if not filename.endswith('.' + file_ext):
                    filename += '.' + file_ext
                with open(os.path.join(plan_chk_filepath, filename), 'w') as f:
                    if file_ext == 'json':
                        f.write(json.dumps(file.json(), indent=4)) 
        

In [ ]:
pts = {col: [] for col in ['MRN', 'Name']}
plans = {col: [] for col in ['ID', 'MRN', 'Name', 'Time', 'Machine', 'Dose Threshold (%)', 'GPR Action Level (%)', 'GPR Tol Level (%)']}
plan_approvals = {col: [] for col in ['Plan ID', 'Time', 'Approver', 'Approver Role']}

In [ ]:
def to_datetime(date, time):
    return datetime.strptime(date + time, '%Y%m%d%H%M%S')

In [61]:
df_dict = {k: [] for k in ['MRN', 'Patient Name', 'Plan', 'Beam Set Number', 'Beam Set Name', 'Beam Number', 'Beam Name', 'Beam Energy (MV)', 'TPS Beam MU', 'M3D Beam MU', 'TPS Beam Dose (cGy)', 'M3D Beam Dose (cGy)', 'Beam Dose Difference (%)', 'Diode Dose for Beam (cGy)', 'External Dose for Beam (cGy)', 'Number of Beam Segments', 'Beam X1 Jaw (cm)', 'Beam X2 Jaw (cm)', 'Beam Y1 Jaw (cm)', 'Beam Y2 Jaw (cm)', 'Beam Wedge', 'Beam MLC', 'Beam Rotation', 'Beam Gantry Angle (\u00b0)', 'Beam Collimator Angle (\u00b0)', 'Beam Couch Angle (\u00b0)', 'Beam Applicator', 'Beam Boli', 'Beam Gantry Clearance (cm)', 'Beam Is Deliverable', 'Beam Fluence Is Combined', 'Beam Fluence Mode']}

for pt in data['patients']:
    for plan in pt['plans']:
        plan_id = plan['request_cid']
        if 'linac_tps' not in plan_id:
            continue
            
        plan = s.get('{}check/details/{}?format=json'.format(serverURL, plan['request_cid'])).json()
        if plan['request']['finished'] != 'clean':
            continue
            
        plans['ID'].append(plan_id)
        plans['Time'].append(datetime.fromtimestamp(plan['request']['finished_timestamp']))
        plans['Machine'].append(plan['request']['machineAliasName_str'])
        plans['Name'].append(plan['request']['planName_str'])
        plans['Dose Threshold (%)'].append(plan['request']['settings']['threshold'])
        plans['GPR Action Level (%)'].append(plan['request']['settings']['gammaAlert_frac'])
        plans['GPR Tol Level (%)'].append(plan['request']['settings']['gammaWarn_frac'])

        plan_info = plan['request']['planInfo_dict']
        plan_approval = plan_info['Approval']
        if plan_approval['ApprovalStatus'] == 'APPROVED':
            plans['Plan Approval Time'].append(to_datetime(plan_approval['ReviewDate'], plan_approval['ReviewTime']))
            plans['Plan Approver'].append(plan_approval['ReviewerName'])  # Fix so it looks up actual name (not CRMCHEALTH_b045z)
        else:
            plans['Plan Approval Time'].append(None)
            plans['Plan Approver'].append(None)

        plans['TPS'].append(plan_info['GeneralEquipment']['ManufacturersModelName'])
        plans['TPS Version'].append(plan_info['GeneralEquipment']['SoftwareVersions'][-1])  # Ignore the RS version that ends with ' (Dicom Export)'

        plans['MD'].append(plan_info['GeneralStudy']['ReferringPhysiciansName'])  # Fix to format name for display
        plans['Sim Date'].append(to_datetime(plan_info['GeneralStudy']['StudyDate'], plan_info['GeneralStudy']['StudyTime']))
        
        pt_info = plan_info['Patient']
        plans['MRN'].append(pt_info['PatientsId'])
        if pt_info['PatientID'] not in pts['MRN']:
            pts['MRN'].append(pt_info['PatientID'])
            pts['Name'].append(pt_info['PatientsName'])  # Fix to format name for display
            pts['DOB'].append(None if pt_info['PatientsBirthDate'] == '' else pt_info['PatientsBirthDate'])  # Better date format?
            pts['Sex'].append(pt_info['PatientsSex'])
        
        for approval in plan['request']['approvals'].values():
            plan_chk_approvals['Plan ID'].append(approval['_id'])
            plan_chk_approvals['Time'].append(datetime.fromtimestamp(approval['added_timestamp']))
            plan_chk_approvals['Approver'].append(approval['full_name']['lastname'] + ', ' + approval['full_name']['firstname'])
            plan_chk_approvals['Approver Role'].append(approval['role'])

        
        # Data from attachments
        attachments = s.get('{}check/details/{}/data?format=json'.format(serverURL, plan['request_cid']))
        

        for beam_set_info in data['data']['beamDose_info'].values():
            for beam_dose_info in beam_set_info.values():
                m3d_mu = beam_dose_info['computedMu']['value']
                m3d_dose = beam_dose_info['computedDose']['value'] * 100  # Convert to cGy
                dose_diff = beam_dose_info['diff']['value']
                diode_dose = beam_dose_info['diodeDose']['value'] * 100
                ext_dose = beam_dose_info['externalDose']['value'] * 100
                fluence_is_combined = beam_dose_info['fluenceCombined_bool']
                
                beam_info = data['data']['beam_info']['beam_num2info_dict'][beam_num]
                beam_name = beam_info['Beam Name']
                appl = beam_info['applicator']['display_str']
                boli = beam_info['boli']
                coll_angle = float(beam_info['collimatorAngle']['display_str']['en'][:-1])
                couch_angle = float(beam_info['couchAngle']['display_str']['en'][:-1])
                gantry_angle = float(beam_info['gantryAngle']['display_str']['en'][:-1])
                collision = beam_info['collision']['display_str']['en']
                is_deliversable = beam_info['deliverable']['display_str'] == 'Yes'
                energy = beam_info['energy']['value']
                fluence_mode = beam_info['fluence_mode']
                jaw_x1 = beam_info['jaws']['x0']['value']
                jaw_x2 = beam_info['jaws']['x1']['value']
                jaw_y1 = beam_info['jaws']['y0']['value']
                jaw_y2 = beam_info['jaws']['y1']['value']
                mlc_type = beam_info['mlcType_str']
                num_seg = beam_info['numBeamOnSegments']['value']
                num_ctrl = beam_info['numControlPoints']['value']
                pt_pos = beam_info['patientSetupPosition_str']
                pt_setup = beam_info['patientSetupTechnique_str']
                rot_type = beam_info['rotationType_str']
                mach_name = beam_info['tdsIdName_str']
                mach_coord = beam_info['tdsInfo_dict']['coordinates']
                clinic = beam_info['tdsInfo_dict']['institutionName_str']
                mlc = beam_info['tdsInfo_dict']['mlc']
                mach_model = beam_info['tdsInfo_dict']['model']
                mach_ref_model = beam_info['tdsInfo_dict']['referenceModel']
                mach_vendor = beam_info['tdsInfo_dict']['vendor']
                wedge = None if beam_info['wedge']['display_str'] == 'None' else beam_info['wedge']['display_str']
                mach_gantry_is_enclosed = beam_info[]
                
                for e, d10 in beam_info['tdsInfo_dict']['energy2depth10dose_dict'].values():
                    for matched_mach in beam_info['tdsInfo_dict']['matchedMachineName_list']:
                
                        df_dict['M3D MU'].append(m3d_mu)
                        df_dict['M3D Beam Dose (cGy)'].append(m3d_dose)
                        df_dict['Dose Difference (%)'].append(dose_diff)
                        df_dict['Diode Dose (cGy)'].append(diode_dose)
                        df_dict['External Dose (cGy)'].append(ext_dose)
                        df_dict['Fluence Is Combined'].append(fluence_is_combined)
                        df_dict['Beam Name'].append(beam_name)
                        df_dict['Applicator'].append(appl if appl != 'None' else None)
                        df_dict['Boli'].append(boli if boli != '' else None)
                        df_dict['Collimator Angle (\u00b0)'].append(coll_angle)
                        df_dict['Couch Angle (\u00b0)'].append(couch_angle)
                        df_dict['Gantry Angle (\u00b0)'].append(gantry_angle)
                        df_dict['Collision'].append(collision)
                        df_dict['Beam Is Deliverable'].append(is_deliverable)  # Add info from 'details'?
                        df_dict['Energy (MV)'].append(energy)
                        df_dict['Fluence Mode'].append(fluence_mode)
                        df_dict['Beam Jaw X1 (cm)'].append(jaw_x1)
                        df_dict['Beam Jaw X2 (cm)'].append(jaw_x2)
                        df_dict['Beam Jaw Y1 (cm)'].append(jaw_y1)
                        df_dict['Beam Jaw Y2 (cm)'].append(jaw_y2)
                        df_dict['Beam MLC Type'].append(mlc_type)
                        df_dict['Number of Beam Segments'].append(num_seg)
                        df_dict['Number of Beam Control Points'].append(num_ctrl)
                        df_dict['Patient Setup Position'].append(pt_pos)
                        df_dict['Patient Setup Technique'].append(pt_setup)
                        df_dict['Beam Rotation Type'].append(rot_type)
                        df_dict['Planning Machine Name'].append(mach_name)
                        df_dict['Planning Machine Coordinate System'].append(mach_coord)

                        df_dict['Planning Machine D10 Energy'].append(e)
                        df_dict['Planning Machine D10 (cGy)'].append(d10 * 100)  # Convert to cGy
                    
                        df_dict['Matched Machine'].append(matched_mach)
                        data['data']['beam_info']['tds_gantry_isEnclodedGantry_bool']
                        data['data']['beam_info']['tds_jaw_isPlaceholderJaw_bool']
                        
                        data['computedIsodose_slices_data']['max_computed_dose_cgy']  #  TPS
                        data['computedIsodose_slices_data']['max_external_dose_cgy']  # M3D
                        
                        num_slices = int(data['ct_dicom']['info']['display_str'].split()[0])
                        data['ct_info']['fromLeftHandedCt_bool']
                        data['ct_info']['patientPosition']
                        
                        
                
    dfs.append(pd.DataFrame(plan_dict))
    return pd.concat([plan_series, results_series])

In [74]:
def split_notes(notes):
    split = notes.split(', ')
    if len(split) == 1:
        return {'Plan Name': notes, 'Beam Set Name': notes}
    return {'Plan Name': split[0], 'Beam Set Name': split[1]}

In [80]:
df = pd.DataFrame.from_dict(data['patients']).explode('plans')
df

,patientId,patientName,plans
0,000354065,Abbott^Krystal^G,"{'created_timestamp': 1643138169.04865, 'messa..."
0,000354065,Abbott^Krystal^G,"{'created_timestamp': 1643138099.084443, 'mess..."
1,000608782,Adcock^Ricky^Delano,"{'created_timestamp': 1639498319.421591, 'mess..."
2,000104761,Akenson^Helen^Elizabeth,"{'created_timestamp': 1635184045.473286, 'mess..."
3,000485348,Aldrich^Donna^F,"{'created_timestamp': 1633968330.695856, 'mess..."
4,000032575,Allen^Gary^Donell,"{'created_timestamp': 1634849421.340708, 'mess..."
5,000423216,Alverson^Earl,"{'created_timestamp': 1635440223.898996, 'mess..."
6,000348289,Anderson^Rebecca^L,"{'created_timestamp': 1627915765.172283, 'mess..."
6,000348289,Anderson^Rebecca^L,"{'created_timestamp': 1627915763.600815, 'mess..."
7,000591486,Andrzejewski^David^B,"{'created_timestamp': 1642713185.364376, 'mess..."


In [78]:
df = pd.DataFrame.from_dict(data['patients']).explode('plans')
df = pd.concat([df, df['plans'].apply(plan_info)], axis=1)
df[['Plan Name', 'Beam Set Name']] = df['notes'].str.split(', ', expand=True)
df['Beam Set Name'] = df.apply(lambda row: row['Plan Name'] if row['Beam Set Name'] is None else row['Beam Set Name'], axis=1)
df['created_timestamp'] = df['created_timestamp'].apply(datetime.fromtimestamp)
df.drop(columns=['notes', 'plans', 'request_cid', 'results'], inplace=True)
df

,patientId,patientName,created_timestamp,message,notes,reason,status,DVH Limit,Gamma,None,Target Coverage,Target Objective,Gantry Collision,Plan Name,Beam Set Name
0,000354065,Abbott^Krystal^G,2022-01-25 13:16:09.048650,Passed all checks.,"VMAT Rectum Bst 540, VMAT Rectum Bo_1",None,ok,ok,ok,ok,ok,none,NaN,VMAT Rectum Bst 540,VMAT Rectum Bo_1
0,000354065,Abbott^Krystal^G,2022-01-25 13:14:59.084443,Passed all checks.,VMAT Rectum 4500,None,ok,ok,ok,ok,ok,none,NaN,VMAT Rectum 4500,VMAT Rectum 4500
1,000608782,Adcock^Ricky^Delano,2021-12-14 10:11:59.421591,Passed all checks.,R Lung,None,ok,ok,ackwarning,ok,ok,none,NaN,R Lung,R Lung
2,000104761,Akenson^Helen^Elizabeth,2021-10-25 12:47:25.473286,Passed all checks.,L Breast,None,ok,ok,ok,NaN,ackwarning,none,ackwarning,L Breast,L Breast
3,000485348,Aldrich^Donna^F,2021-10-11 11:05:30.695856,Passed all checks.,Esophagus VMAT,None,ok,ok,ok,ok,ackerror,none,NaN,Esophagus VMAT,Esophagus VMAT
4,000032575,Allen^Gary^Donell,2021-10-21 15:50:21.340708,Passed all checks.,GBM,None,ok,ackerror,ok,ok,ok,none,NaN,GBM,GBM
5,000423216,Alverson^Earl,2021-10-28 11:57:03.898996,Passed all checks.,Prostate,None,ok,ok,ok,ok,ok,none,NaN,Prostate,Prostate
6,000348289,Anderson^Rebecca^L,2021-08-02 09:49:25.172283,Passed all checks.,"3F R Chest Wall, R Sclav",None,ok,ok,ok,ok,none,none,NaN,3F R Chest Wall,R Sclav
6,000348289,Anderson^Rebecca^L,2021-08-02 09:49:23.600815,Passed all checks.,"3F R Chest Wall, R Chest Wall",None,ok,ok,ok,ok,none,none,NaN,3F R Chest Wall,R Chest Wall
7,000591486,Andrzejewski^David^B,2022-01-20 15:13:05.364376,Passed all checks.,GBM,None,ok,ackerror,ok,ok,ok,none,NaN,GBM,GBM


In [6]:
df = pd.json_normalize(data)
df.explode(df.columns[0])

,patients
0,"{'patientId': '000354065', 'patientName': 'Abb..."
0,"{'patientId': '000608782', 'patientName': 'Adc..."
0,"{'patientId': '000104761', 'patientName': 'Ake..."
0,"{'patientId': '000485348', 'patientName': 'Ald..."
0,"{'patientId': '000032575', 'patientName': 'All..."
0,"{'patientId': '000423216', 'patientName': 'Alv..."
0,"{'patientId': '000348289', 'patientName': 'And..."
0,"{'patientId': '000591486', 'patientName': 'And..."
0,"{'patientId': '000474089', 'patientName': 'Atn..."
0,"{'patientId': '000226508', 'patientName': 'Bak..."
